### Welcome to the ProtoSyn.jl examples

# 4 - Energy Calculation

When comparing two or more molecular structures, it's useful to have a way to measure the fitness (or how "real" a structure is). This is often achieved by applying an energy funcion: a set of energy component calculations summed to return a fitness gauge. In ProtoSyn.jl, this is easily achieved by applying the default `EnergyFunction` or creating a new or custom function. For this example, we will load the 2A3D proteic structure.

In [33]:
using ProtoSyn

In [18]:
using ProtoSyn
ProtoSyn.acceleration.active = ProtoSyn.SIMD_1

ProtoSyn.SIMD_1

In [19]:
pose = ProtoSyn.Peptides.load("data/2a3d.pdb")
ProtoSyn.Peptides.assign_default_atom_names!(pose)
ProtoSyn.Peptides.Calculators.Electrostatics.assign_default_charges!(pose);

In [20]:
energy_function = ProtoSyn.Common.default_energy_function()

🗲  Energy Function (7 components):
+----------------------------------------------------------------------+
| Index | Component name                                | Weight (α)   |
+----------------------------------------------------------------------+
| 1     | TorchANI_ML_Model                             |   3.00e-04   |
| 2     | All_Atom_Clash_Rest                           |       1.00   |
| 3     | Bond_Distance_Rest                            |       0.05   |
| 4     | Hydrogen_Bonds                                |       0.50   |
| 5     | Coulomb                                       |       0.60   |
| 6     | GB_Solvation                                  |       0.50   |
| 7     | SASA                                          |   3.00e-03   |
+----------------------------------------------------------------------+
 ● Update forces: false
 ● Selection: Set
 └── TrueSelection (Atom)


In [21]:
energy_function(pose)

10.94244275999175

Since our energy function had multiple energy components, we can analyze each individual contribution in the `pose.state.e` dictionary.

In [22]:
pose.state.e

Dict{Symbol, Float64} with 8 entries:
  :Total               => 10.9424
  :TorchANI_ML_Model   => -0.00295364
  :SASA                => 4.176
  :GB_Solvation        => -4.75801
  :Coulomb             => 0.092721
  :Hydrogen_Bonds      => -4.66042
  :All_Atom_Clash_Rest => 16.0951
  :Bond_Distance_Rest  => 2.08107e-5

By setting the `update_forces` flag to true, when calling the energy function, we can also calculate the forces felt on each atom of the Pose.

In [23]:
energy_function(pose, update_forces_overwrite = true)

LoadError: TypeError: in keyword argument v, expected Union{Nothing, Tuple{Float64, Float64, Float64}}, got a value of type SIMD.Vec{4, Float64}

In [ ]:
pose.state.f

3×1140 Matrix{Float64}:
 -1.0026e-5   1.14238e-5  -0.0166671  …  -0.365434   0.568672  -0.551533
 -3.64531e-6  7.01899e-7  -0.0198531      0.262988   0.150171  -0.200869
 -1.6739e-6   5.0517e-6   -0.0110529     -0.148427  -0.188661   0.19224

The energy function is completly modifiable, the user can add or remove components, as well as change the weight bias given to each one, as exemplified bellow. 

In [ ]:
energy_function["Caterpillar_Solvation"].α = 0.03;
energy_function

LoadError: KeyError: key "Caterpillar_Solvation" not found

In [ ]:
energy_function(pose)

-0.10164740027243191

As we can see, the resulting energy is now different. Besides altering the `α` weight bias, some `EnergyFunctionComponent` instances have a set of settings, specific for each one, that can be fine tuned. for example, we can set the bond distance restraint to a lower distance.

In [ ]:
energy_function["Bond_Distance_Restraint"].settings[:x0] = 0.2
energy_function["Bond_Distance_Restraint"]

          Name : Bond_Distance_Restraint
    Weight (α) : 1.0
 Update forces : true
       Setings :
            :x0 => 0.2


In [ ]:
energy_function(pose)

365.8297953062032

Which, of course, results in an absurd increase in energy of the system.

## Conclusion

In this brief example we took a look at how to gauge the fitness of a structure, using the default Energy Function instance of ProtoSyn, as well as how to modify and fine tune each of the individual energy function component instances that compose the energy function.